In [22]:
from __future__ import division
from __future__ import print_function
import pandas as pd
import numpy as np
import os
import sys
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
import re
from sklearn.preprocessing import RobustScaler
import scipy.stats as ss
# from sklearn.metrics import f1_score
# from sklearn.metrics import accuracy_score
from sklearn import metrics
import scipy.io
import os
import sys
from time import time
import scipy.stats as ss
from sklearn.preprocessing import RobustScaler
# temporary solution for relative imports in case pyod is not installed
# if pyod is installed, no need to use the following line
sys.path.append(
    os.path.abspath(os.path.join(os.path.dirname("__file__"), '..')))
from numpy import percentile
import matplotlib.pyplot as plt
import matplotlib.font_manager

# Import all models
from pyod.models.abod import ABOD
from pyod.models.cblof import CBLOF
from pyod.models.hbos import HBOS
from pyod.models.iforest import IForest
from pyod.models.knn import KNN
from pyod.models.copod import COPOD
from pyod.models.lof import LOF
from pyod.models.ocsvm import OCSVM
from pyod.models.pca import PCA
from pyod.models.lscp import LSCP
from IPython.display import display
import time


# dataset path
data_path = "/Users/kadima/experiment_any/anomaly-detection/datasets_resend/"




# 1. Data and Classifier Preparation

In [23]:
training_dict = dict()
label_dict = dict()

for root, path, files in os.walk("../datasets_resend/"):
    for file in files:
        if file.endswith("txt"):
            if "label" not in file:
                with open(root+file,'r') as d:
                    data = d.readlines()
                    data = [x.split() for x in data]
                    data = [[float(i) for i in x] for x in data]
                training_dict[file[:-9]] = np.asarray(data).astype(float)
            else:
                with open(root+file,'r') as d:
                    label = d.readlines()
                    label = np.asarray([1-int(x[0]) for x in label ])
                label_dict[file[:-10]] = label

In [24]:
random_state = np.random.RandomState(10)
outliers_fraction = 0.4
detector_list = [LOF(n_neighbors=5), LOF(n_neighbors=10), LOF(n_neighbors=15),
                 LOF(n_neighbors=20), LOF(n_neighbors=25), LOF(n_neighbors=30),
                 LOF(n_neighbors=35), LOF(n_neighbors=40), LOF(n_neighbors=45),
                 LOF(n_neighbors=50)]


# initialize a set of detectors for LSCP
classifiers = {
#     'Angle-based Outlier Detector (ABOD)':
#         ABOD(contamination=outliers_fraction),

    'Histogram-base Outlier Detection (HBOS)': HBOS(
        contamination=outliers_fraction),
    'Isolation Forest': IForest(contamination=outliers_fraction,
                                random_state=random_state, n_estimators=280),
    'K Nearest Neighbors (KNN)': KNN(
        contamination=outliers_fraction),
    'Average KNN': KNN(method='mean',
                       contamination=outliers_fraction),
    'Local Outlier Factor (LOF)':
        LOF(n_neighbors=35, contamination=outliers_fraction),
#     'One-class SVM (OCSVM)': OCSVM(contamination=outliers_fraction),
    'Principal Component Analysis (PCA)': PCA(
        contamination=outliers_fraction, random_state=random_state),
    "COPOD": COPOD(),
    
    'Locally Selective Combinatio (LSCP)': LSCP(
    detector_list)
}

classifiers2 = dict(zip(['lof_'+str(i) for i in range(5,51,5)],[LOF(n_neighbors=x) for x in range(5,51,5)]))

names = []
for i, clf in enumerate(classifiers.keys()):
    names.append(clf)



# Data Check

outlier label = 0

In [25]:
for fname, data in training_dict.items():
    df_tmp = pd.DataFrame(data)
    if df_tmp.duplicated().sum() >= 1:
        print(fname)

WBC_v01
thyroid
breastw
Lymphography_withoutdupl_catremoved
satimage-2
Ionosphere_withoutdupl_norm
Glass_withoutdupl_norm
letter
mammography
Cardiotocography_withoutdupl_22
optdigits
vowels


# 2. Disagreement

步骤：
1. 算出各detector的分数并进行归一化
2. 选出一个main detector
3. 算出每个instance的disagreement
4. 如果大于某个阈值（平均disagreement？，或者 大于 mad disagreement），则在该分数上去平均outlier scores（或者取max）

In [26]:
def get_detectors_scores(X,y,clfs):
    trained_clfs = []
    clf_scores_dict = dict()
    # Fit the model
    for i, (clf_name, clf) in enumerate(clfs.items()):
        print(clf_name)
        start_time = time.time()
        clf.fit(X)
        trained_clfs.append((clf_name, clf))
        scores_pred = clf.decision_function(X)
        standard_scores= RobustScaler().fit_transform(np.reshape(scores_pred,(-1,1)))
        clf_scores_dict[clf_name] = standard_scores.reshape(-1)
        end_time = time.time()
        print("cost",(end_time-start_time)//60,'minutes')
        print('-'*20)
    return pd.DataFrame.from_dict(clf_scores_dict)


def get_disagreement(score_df, return_rank = False):
    std_rows = score_df.std(axis=1)
    return std_rows

def get_mad_threshold(scores):
    median_ = np.median(scores)
    mad_3 = 3*1.4826*np.median(np.abs(scores-median_))
    return mad_3

def scoring_strategy(df, main_detector_name, strategy = 'max'):
    disagreement_scores = get_disagreement(df)
    mad_ = get_mad_threshold(disagreement_scores)
    classifier_names = df.columns
    df['disagreement'] = disagreement_scores
    if strategy == 'max':
        df['ensemble'] = df.apply(lambda x: np.max([x[p] for p in classifier_names]) 
           if x['disagreement'] >= mad_ 
           else x[main_detector_name],
                 axis=1)
        
    elif strategy == 'mean':
        df['ensemble'] = df.apply(lambda x: np.mean([x[p] for p in classifier_names]) 
           if x['disagreement'] >= mad_ 
           else x[main_detector_name],
                 axis=1)
        
    return df


def softmax(scores_array):
    weights = scores_array - max(scores_array)
    s = np.exp(weights).sum()
    weights = np.exp(weights)/s
    smooth_scores = weights * scores_array
    return sum(smooth_scores)
    
    
    

In [35]:
result_dict = dict()
file_auc_dict = dict()
for fname in training_dict.keys():
    print(fname)
    result_dict[fname] = get_detectors_scores(training_dict[fname], label_dict[fname], classifiers2)
    new_df = scoring_strategy(result_dict[fname], 'lof_10', 'min')
    file_auc_dict[fname] = pd.DataFrame(new_df.apply(lambda x:  metrics.roc_auc_score(label_dict[fname],x)))
    print("="*45)

Shuttle_withoutdupl_v01
lof_5
cost 0.0 minutes
--------------------
lof_10
cost 0.0 minutes
--------------------
lof_15
cost 0.0 minutes
--------------------
lof_20
cost 0.0 minutes
--------------------
lof_25
cost 0.0 minutes
--------------------
lof_30
cost 0.0 minutes
--------------------
lof_35
cost 0.0 minutes
--------------------
lof_40
cost 0.0 minutes
--------------------
lof_45
cost 0.0 minutes
--------------------
lof_50
cost 0.0 minutes
--------------------
Stamps_withoutdupl_09
lof_5
cost 0.0 minutes
--------------------
lof_10
cost 0.0 minutes
--------------------
lof_15
cost 0.0 minutes
--------------------
lof_20
cost 0.0 minutes
--------------------
lof_25
cost 0.0 minutes
--------------------
lof_30
cost 0.0 minutes
--------------------
lof_35
cost 0.0 minutes
--------------------
lof_40
cost 0.0 minutes
--------------------
lof_45
cost 0.0 minutes
--------------------
lof_50
cost 0.0 minutes
--------------------
InternetAds_withoutdupl_norm_19
lof_5
cost 0.0 minutes
-

cost 0.0 minutes
--------------------
lof_10
cost 0.0 minutes
--------------------
lof_15
cost 0.0 minutes
--------------------
lof_20
cost 0.0 minutes
--------------------
lof_25
cost 0.0 minutes
--------------------
lof_30
cost 0.0 minutes
--------------------
lof_35
cost 0.0 minutes
--------------------
lof_40
cost 0.0 minutes
--------------------
lof_45
cost 0.0 minutes
--------------------
lof_50
cost 0.0 minutes
--------------------
Ionosphere_withoutdupl_norm
lof_5
cost 0.0 minutes
--------------------
lof_10
cost 0.0 minutes
--------------------
lof_15
cost 0.0 minutes
--------------------
lof_20
cost 0.0 minutes
--------------------
lof_25
cost 0.0 minutes
--------------------
lof_30
cost 0.0 minutes
--------------------
lof_35
cost 0.0 minutes
--------------------
lof_40
cost 0.0 minutes
--------------------
lof_45
cost 0.0 minutes
--------------------
lof_50
cost 0.0 minutes
--------------------
Glass_withoutdupl_norm
lof_5
cost 0.0 minutes
--------------------
lof_10
cost 0

cost 0.0 minutes
--------------------
lof_10
cost 0.0 minutes
--------------------
lof_15
cost 0.0 minutes
--------------------
lof_20
cost 0.0 minutes
--------------------
lof_25
cost 0.0 minutes
--------------------
lof_30
cost 0.0 minutes
--------------------
lof_35
cost 0.0 minutes
--------------------
lof_40
cost 0.0 minutes
--------------------
lof_45
cost 0.0 minutes
--------------------
lof_50
cost 0.0 minutes
--------------------
speech
lof_5
cost 0.0 minutes
--------------------
lof_10
cost 0.0 minutes
--------------------
lof_15
cost 0.0 minutes
--------------------
lof_20
cost 0.0 minutes
--------------------
lof_25
cost 0.0 minutes
--------------------
lof_30
cost 0.0 minutes
--------------------
lof_35
cost 0.0 minutes
--------------------
lof_40
cost 0.0 minutes
--------------------
lof_45
cost 0.0 minutes
--------------------
lof_50
cost 0.0 minutes
--------------------
satellite
lof_5
cost 0.0 minutes
--------------------
lof_10
cost 0.0 minutes
--------------------
lo

In [34]:
cnt_ = 0
for fname in file_auc_dict.keys():
    if file_auc_dict[fname].loc['ensemble',0] >= (file_auc_dict[fname].loc[list(classifiers2.keys()),:]).max()[0]:
        print(fname)
        print("ensemble AUC",file_auc_dict[fname].loc['ensemble',0])
        print('Best classifier auc', (file_auc_dict[fname].loc[list(classifiers2.keys()),:]).max()[0])
        cnt_ += 1
        print(cnt_,"\n\n")

Shuttle_withoutdupl_v01
ensemble AUC 0.9613846153846154
Best classifier auc 0.9473076923076924
1 


InternetAds_withoutdupl_norm_19
ensemble AUC 0.7019406051042063
Best classifier auc 0.6651367878326169
2 


cover
ensemble AUC 0.61252299932156
Best classifier auc 0.5769668596811339
3 


Arrhythmia_withoutdupl_46
ensemble AUC 0.7653191150724176
Best classifier auc 0.7621956071940157
4 


satimage-2
ensemble AUC 0.6034714918962483
Best classifier auc 0.575933970887432
5 


Glass_withoutdupl_norm
ensemble AUC 0.8823848238482385
Best classifier auc 0.8563685636856369
6 


WPBC_withoutdupl_norm
ensemble AUC 0.5290968014654079
Best classifier auc 0.5228969987318586
7 




In [29]:
len(file_auc_dict)

37

In [30]:
file_auc_dict

{'Shuttle_withoutdupl_v01':                      0
 lof_5         0.643154
 lof_10        0.942462
 lof_15        0.938615
 lof_20        0.947231
 lof_25        0.947308
 lof_30        0.923462
 lof_35        0.911538
 lof_40        0.909692
 lof_45        0.917385
 lof_50        0.935000
 disagreement  0.962000
 ensemble      0.962462,
 'Stamps_withoutdupl_09':                      0
 lof_5         0.459965
 lof_10        0.543481
 lof_15        0.597035
 lof_20        0.679194
 lof_25        0.705710
 lof_30        0.703205
 lof_35        0.702892
 lof_40        0.707590
 lof_45        0.714375
 lof_50        0.725650
 disagreement  0.564360
 ensemble      0.546821,
 'InternetAds_withoutdupl_norm_19':                      0
 lof_5         0.575589
 lof_10        0.581546
 lof_15        0.626031
 lof_20        0.655428
 lof_25        0.665137
 lof_30        0.662535
 lof_35        0.657985
 lof_40        0.651030
 lof_45        0.646132
 lof_50        0.646069
 disagreement  0.689872

### MAD with max
```python
WBC_v01
ensemble AUC 0.5522522522522523
Best classifier auc 0.3921171171171171
1 


breastw
ensemble AUC 0.7015435938030081
Best classifier auc 0.6335896566022088
2 


Lymphography_withoutdupl_catremoved
ensemble AUC 0.37617370892018775
Best classifier auc 0.2764084507042253
3
```


In [62]:
a = np.asarray([3,5,1,10,22])
b = np.tile(a, (5,1) )
c = np.tile(a, (5,1) ).T
print("b")
print(b)
print()
print("c")
print(c)

d = np.asarray([0,1,2,3,4])
e = np.tile(d, (5,1))
f = np.tile(d, (5,1)).T
print()
print("e")
print(e)
print()
print("f")
print(f)


b
[[ 3  5  1 10 22]
 [ 3  5  1 10 22]
 [ 3  5  1 10 22]
 [ 3  5  1 10 22]
 [ 3  5  1 10 22]]

c
[[ 3  3  3  3  3]
 [ 5  5  5  5  5]
 [ 1  1  1  1  1]
 [10 10 10 10 10]
 [22 22 22 22 22]]

e
[[0 1 2 3 4]
 [0 1 2 3 4]
 [0 1 2 3 4]
 [0 1 2 3 4]
 [0 1 2 3 4]]

f
[[0 0 0 0 0]
 [1 1 1 1 1]
 [2 2 2 2 2]
 [3 3 3 3 3]
 [4 4 4 4 4]]


In [58]:
s = np.random.random((100, 5))

In [63]:
(s[c,e] - s[b, e]).shape

(5, 5)

In [ ]:
def matrixize_disagreement(x_series, rank_matrix):
    

In [3]:
import numpy as np
s = np.round(np.random.random((5,3)),2) * 100
s

array([[28., 11., 75.],
       [41., 39., 32.],
       [39., 83., 63.],
       [ 5., 33., 95.],
       [18.,  3., 42.]])

In [11]:
def sum_except_mid(x_series):
    new_one = np.tile(x_series, 3)
    new_one = np.sum(new_one, axis = 1)
    print(new_one.shape)
    return new_one

In [13]:
np.apply_along_axis(sum_except_mid, 1, s)

AxisError: axis 1 is out of bounds for array of dimension 1